# Does SSA (Sistem Satu Arah - Unidirectional Road System) implementation on some road segments in Depok city make any improvement to the traffic condition of the city?

- Comparing the traffic conditions (in terms of travel time) between various locations in Depok city before (< July 2017) and after (> July 2017) the implementation of SSA
- Using HERE REST API to derive past traffic conditions

### Limitations
Apparently HERE REST API only provides precise historical travel times from 1 January 2018 onward. Travel times before that time are averaged out and categorised into 'morning travel' and 'evening travel'.

# Data input

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import json
import urllib.request
import sys
import osmnx as ox
from shapely.geometry import Point, Polygon
import gmplot
from here_api import *

In [2]:
#list koordinat
points = [Point(106.831913, -6.377133), #pertigaan juanda-margonda arah ramanda
          Point(106.831768, -6.377069), #pertigaan juanda-margonda arah margo city
          Point(106.825930, -6.389564), #pertigaan ramanda arah terminal depok
          Point(106.825812, -6.389415), #pertigaan ramanda arah depok mall
          Point(106.814649, -6.392354), #pasar baru jln nusantara
          Point(106.812714, -6.399052), #jln raya sawangan
          Point(106.813877, -6.397228), #rumah (sarikaya)
          Point(106.813627, -6.399058), #dewi sartika, perlimaan, lajur lama
          Point(106.813580, -6.399127), #dewi sartika, perlimaan, lajur baru
          Point(106.818997, -6.399890), #jln kartini - dewi sartika, arah margonda
          Point(106.819098, -6.399924), #jln kartini - dewi sartika, arah citayam
          Point(106.823846, -6.393196), #ITC depok
          Point(106.824034, -6.393209), #sebrang ITC depok
         ]

In [3]:
points_name = ['01 pertigaan juanda-margonda arah ramanda',
          '02 pertigaan ramanda arah terminal depok',
          '03 pertigaan ramanda arah depok mall',
          '04 pasar baru jln nusantara',
          '05 jln raya sawangan',
          '06 rumah (sarikaya)',
          '07 dewi sartika, perlimaan, lajur lama',
          '08 dewi sartika, perlimaan, lajur baru',
          '09 jln kartini - dewi sartika, arah margonda',
          '10 jln kartini - dewi sartika, arah citayam',
          '11 ITC depok',
          '12 sebrang ITC depok']

In [4]:
times2018_mon = ['2018-02-05T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2018_wed = ['2018-02-07T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2018_fri = ['2018-02-09T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2018_sat = ['2018-02-10T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]

times2017_mon = ['2017-02-06T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2017_wed = ['2017-02-08T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2017_fri = ['2017-02-10T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]
times2017_sat = ['2017-02-11T{}:00:00'.format(x) for x in ['07', '08', '09', '17', '18', '19']]

In [5]:
times2018_fri

['2018-02-09T07:00:00',
 '2018-02-09T08:00:00',
 '2018-02-09T09:00:00',
 '2018-02-09T17:00:00',
 '2018-02-09T18:00:00',
 '2018-02-09T19:00:00']

# Draw isoline
Isoline : line that defines an area that can be reached within a given time from a given point on a map

Results are saved directly into a separate HTML file

In [6]:
#coba isoline 2018
start = points[4]
app_code = ''
app_id = ''
dep_hour = times2018_sat[5]
input_range=600 #in seconds

test_poly1=get_isoline(start, dep_hour, app_code, app_id, input_range)
test_poly_pic=draw_isoline(center=start, range_=14, poly=test_poly1)
test_poly_pic.draw('map_2018.html')

In [7]:
#coba isoline 2017
start = points[4]
app_code = ''
app_id = ''
dep_hour = times2017_sat[5]
input_range=600 #in seconds

test_poly2=get_isoline(start, dep_hour, app_code, app_id, input_range)
test_poly_pic=draw_isoline(center=start, range_=14, poly=test_poly2)
test_poly_pic.draw('map_2017.html')

In [8]:
test_poly_pic_compare = draw_isoline_compare(center=start, range_=14, poly1=test_poly1, poly2=test_poly2)
test_poly_pic_compare.draw('map_2017_2018.html')

# CSV generation

In [9]:
times_pairs = [(times2018_mon, times2017_mon),
               (times2018_wed, times2017_wed),
               (times2018_fri, times2017_fri),
               (times2018_sat, times2017_sat)]

In [11]:
app_code = ''
app_id = ''

gdf_master = gpd.GeoDataFrame()

for times_pair in times_pairs:
    for p1 in range(12): #for each starting position
        for p2 in range(12): #for each ending position
            if p1 != p2:
                start = points[p1]
                end = points[p2]

                #2018 data
                for t in range(6): #for each time of the day
                    dep_hour = times_pair[0][t]
                    if t==0:
                        gdf = get_df(start=start, end=end, time=dep_hour, app_code=app_code, app_id=app_id)
                        gdf['clock_time'] = dep_hour
                    else:
                        gdf2 = get_df(start=start, end=end, time=dep_hour, app_code=app_code, app_id=app_id)
                        gdf2['clock_time'] = dep_hour
                        gdf = gdf.append(gdf2)
                gdf['From'] = points_name[p1]
                gdf['To'] = points_name[p2]
                gdf_master = gdf_master.append(gdf)

                #2017 data
                for t in range(6): #for each time of the day
                    dep_hour = times_pair[1][t]
                    if t==0:
                        gdf = get_df(start=start, end=end, time=dep_hour, app_code=app_code, app_id=app_id)
                        gdf['clock_time'] = dep_hour
                    else:
                        gdf2 = get_df(start=start, end=end, time=dep_hour, app_code=app_code, app_id=app_id)
                        gdf2['clock_time'] = dep_hour
                        gdf = gdf.append(gdf2)
                gdf['From'] = points_name[p1]
                gdf['To'] = points_name[p2]
                gdf_master = gdf_master.append(gdf)

In [12]:
gdf_master.to_csv('data_2017_2018.csv')

In [13]:
len(gdf_master)

6336

# analysis

In [46]:
import pandas as pd
import numpy as np
import sys
from shapely.geometry import Point, Polygon
import datetime
import calendar

df = pd.read_csv('data_2017_2018.csv')

In [48]:
def get_day(dt):
    year, month, day = (int(x) for x in dt.split('-'))
    day_no = datetime.date(year, month, day).weekday()
    return calendar.day_name[day_no]

In [49]:
df['year'] = df['clock_time'].apply(lambda t: int(t[:4]))
df['time_of_day'] = df['clock_time'].apply(lambda t: 'Morning' if int(t[11:13]) < 10 else 'Evening')
df['day'] = df['clock_time'].apply(lambda t: get_day(t[:10]))

In [52]:
df.tail(15)

,Unnamed: 0,distance,time,geometry,clock_time,From,To,year,time_of_day,day
6321,0,945,126,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2017-02-11T17:00:00,12 sebrang ITC depok,"10 jln kartini - dewi sartika, arah citayam",2017,Evening,Saturday
6322,0,945,126,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2017-02-11T18:00:00,12 sebrang ITC depok,"10 jln kartini - dewi sartika, arah citayam",2017,Evening,Saturday
6323,0,945,126,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2017-02-11T19:00:00,12 sebrang ITC depok,"10 jln kartini - dewi sartika, arah citayam",2017,Evening,Saturday
6324,0,932,126,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2018-02-10T07:00:00,12 sebrang ITC depok,11 ITC depok,2018,Morning,Saturday
6325,0,932,139,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2018-02-10T08:00:00,12 sebrang ITC depok,11 ITC depok,2018,Morning,Saturday
6326,0,932,150,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2018-02-10T09:00:00,12 sebrang ITC depok,11 ITC depok,2018,Morning,Saturday
6327,0,932,180,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2018-02-10T17:00:00,12 sebrang ITC depok,11 ITC depok,2018,Evening,Saturday
6328,0,932,178,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2018-02-10T18:00:00,12 sebrang ITC depok,11 ITC depok,2018,Evening,Saturday
6329,0,932,170,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2018-02-10T19:00:00,12 sebrang ITC depok,11 ITC depok,2018,Evening,Saturday
6330,0,932,124,"LINESTRING (106.8239155 -6.3932365, 106.823812...",2017-02-11T07:00:00,12 sebrang ITC depok,11 ITC depok,2017,Morning,Saturday
